Imports

In [8]:
## Basic module imports
import pandas as pd
import os
import numpy as np
import torchvision.transforms as torchtransforms
from PIL import Image, ImageDraw
import json
import shutil

## Maibi_cv imports
import lib.detection.transforms as transforms
from lib.detection.engine import evaluate
from lib.detection.engine import train_one_epoch
from lib.detection.engine import get_coco_api_from_dataset
import lib.detection.utils as utils_maibi_cv

## Torch imports
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter

## Visualization imports
from IPython.display import display

## Torchvision imports 
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

## Custom developments 
from towermonitoring_lib import TowerDataSet2
from towermonitoring_lib import LabelMapping

from sklearn.metrics import confusion_matrix


Method 0: Draw Bounding Boxes

In [9]:
def draw_bounding_boxes (image, bounding_boxes): 
    to_pil_image = torchtransforms.ToPILImage()
    img_pil = to_pil_image(image)
    draw = ImageDraw.Draw(img_pil)
        
    for box in bounding_boxes:
        display(box)
        draw.rectangle(box.numpy(), outline="red", width=3)
   
    img_pil.show()
    
    

Method 1: Get predictions from a Model

In [10]:
def get_predictions (model, data_loader, device):
    model.eval()
    predicted_labels = []
    with torch.no_grad(): 
        for images, _ in data_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predicted_labels.extended(predicted.cpu().numpy())
    return predicted_labels
            

Step 1: Create TowerDataSet2

In [11]:
RELPATH_LABELSFILE = ".\\data5\\patches_faster_rcnn_network\\all_patches_annotated.csv"
ABSPATH_LABELSFILE = os.path.abspath(RELPATH_LABELSFILE)

tfm = torchtransforms.ToTensor()

train_tfm = transforms.Compose([
    transforms.ToTensor(),  # converts the image, a PIL image, into a PyTorch Tensor
    #Trans.RandomHorizontalFlip(0.5)  # randomly flip the training images
])

fulldataset = TowerDataSet2.TowerDataset2(RELPATH_LABELSFILE, tfm)

Step 2: Visual Inspection Bounding Boxes TowerDataSet2

In [12]:
#for imagename, img, target in fulldataset:
#    print(imagename)
#    draw_bounding_boxes(imagename, img, target["boxes"])
#    input("Press key to show next image...")
img, target = fulldataset.getitem_name('210_011.63_015.35_H1_035_37_original.JPG')
draw_bounding_boxes(img, target["boxes"])


tensor([ 82.6770, 109.5720, 134.4747, 156.3891])

tensor([ 68.7315, 159.3774,  89.6498, 223.1284])

tensor([133.4786, 141.4475, 147.4241, 184.2802])

tensor([ 90.6459, 158.3813, 119.5331, 210.1790])

tensor([114.5525, 194.2412, 132.4825, 238.0700])

tensor([ 79.6887, 445.2607, 107.5798, 512.0000])

Step 3: Split DataSet & Create DataLoaders

In [13]:
dataset_size = len(fulldataset)

train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset, val_dataset = random_split(fulldataset, [train_size, val_size])

data_loader_train = DataLoader(
    train_dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils_maibi_cv.collate_fn
)

data_loader_val = DataLoader(
    val_dataset, batch_size=2, shuffle=False, num_workers=4,
    collate_fn=utils_maibi_cv.collate_fn
)

Step 4: Create and initialize the model

In [14]:
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# our dataset has two classes only - background and product
num_classes = 2
#in_features = model.roi_heads.box_predictor.cls_score.in_features
#model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = StepLR(optimizer,
                      step_size=3,
                      gamma=0.1)

Step 5: Train the model

In [15]:
num_epochs = 5

writer = SummaryWriter()

coco = get_coco_api_from_dataset(data_loader_val.dataset)

#if __name__ == '__main__':
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader_train,
                    device, epoch, writer=writer)

    # update the learning rate
    print(f'Epoch {epoch} training done')
    lr_scheduler.step()
    
    # evaluate on the validation dataset
    evaluate(model, data_loader_val, device, epoch, coco, writer=writer)
    
#save the results 
torch.save(model.state_dict(), '.\\data5\\patches_faster_rcnn_network\\fasterrcnn_model_weights_20240518')

creating index...
index created!


Epoch 0 training done


Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.497
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.054
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.180
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.270
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.120
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.309
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.329
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.277
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.342
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

Epoch 1 training done


Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.198
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.594
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.254
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.125
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.335
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.372
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.319
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.379
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

...

Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.238
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.601
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.089
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.153
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.367
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.385
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.327
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.401
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

Epoch 3 training done


Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.256
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.639
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.133
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.234
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.267
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.303
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.147
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.382
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.389
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.350
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.404
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

Epoch 4 training done


Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.255
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.619
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.128
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.291
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.145
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.372
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.379
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.342
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.391
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

Step 6: Create a Confusion Matrix

In [22]:
predicted_labels = get_predictions(model, data_loader_val, device)

display(predicted_labels)

AttributeError: 'tuple' object has no attribute 'to'